# Hochschule Bonn-Rhein-Sieg

# Probabilistic Reasoning, WS21

# Assignment 05

Instructions for submission :
- Please restart and run all cells before submitting 
- Make sure your user name is correct
- No need to submit a pdf file, only the ipython is sufficient


Instruction for assignment :

- This is a two week long assignment,
- The assignment uses the "pgmpy" library which can be used to represent and work with bayesian networks 
- Read the instruction in https://github.com/pgmpy/pgmpy to install the library
- In the first week it is recommended to follow the working examples from https://github.com/pgmpy/pgmpy/tree/dev/examples to fammilarise yourself with the library 
- Second week can be used to finish the assignment 
 

Good luck !!


In [1]:
import numpy as np
import IPython

from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD

### Student user name: nmursa2s



### Exercise 1: Variable elimination [40 points]

This exercise is rather simple to describe: you have to implement the variable elimination algorithm for answering queries in a Bayesian network. Now, implementing variable elimination from scratch is actually a lot of work, so most of it has therefore been done by us; however, you will have to fill in a few key functions that will make the skeleton code work correctly.

In the cell below, we have two main classes, namely *Factor* and *ExactInferenceEngine*.

1. *Factor* is a data structure for storing the factors used in variable elimination. The class exposes two methods: *multiply*, which takes the calling object as a first parameter and another factor as a second parameter, and *sum_out*, which takes the calling object as a first parameter and the name of a variable as a second parameter.
2. *ExactInferenceEngine* is an interface for performing inference using variable elimination. The class exposes only one method, namely *perform_inference*, which takes the name of a query variable as a first parameter (we assume that we only have one query variable) and a dictionary of evidence variables and values as a second argument. The class has other methods as well, but they are all private, namely (i) *\_\_find_dependency_levels*, which sorts the terms of the joint distribution represented by a Bayesian network based on the variables that we have to sum out, (ii) *\_\_create_factors*, which creates the initial set of factors before we start performing inference, and (iii) *\_\_multiply_factors*, which takes a list of still active factors, a factor that acts as a placeholder of the current result of the inference process, a list of dependency levels as returned by *\_\_find_dependency_levels*, and the current level in the summation. The functionality of the private methods is fully implemented, so you don't have to implement anything there.

**Your main task is to implement the methods *Factor.multiply*, *Factor.sum_out*, and part of the functionality of *ExactInferenceEngine.perform_inference***. In particular, *Factor.multiply* and *Factor.sum_out* are almost fully empty, so you have to implement the functionality pretty much completely; on the other hand, *ExactInferenceEngine.perform_inference* is almost fully written, but you have to implement the main inference loop (which amounts to you calling the appropriate methods with the appropriate parameters). The parts where you have to implement your code are clearly marked; please leave the rest of the code as it is. The code has quite a lot of comments, so you should in principle be able to understand how everything works and what you're expected to do.

In [2]:
class Factor(object):
    def __init__(self, variables=[], values=[], probabilities=[]):
        '''Defines a new factor of the variables in 'variables'.

        Keyword arguments:
        variables -- A list of variables in the factor.
        values -- A 2D list of values of the factor variables.
        probabilities -- A 2D list of probabilities
                         corresponding to the factor variables;
                         'probabilities' should be aligned with 'values', i.e.
                         'probabilities[i]' should result from 'values[i]'.
        '''
        self.variables = variables
        self.values = values
        self.probabilities = probabilities

    def multiply(self, other):
        '''Multiplies 'self' with 'other'.

        Keyword arguments:
        other -- A 'Factor' object.

        Returns:
        new_factor -- A 'Factor' object representing the product of
                      the factors 'self' and 'other'.
        '''

        #we create the set of variables in the new factor
        new_variables = list(set(self.variables).union(set(other.variables)))
        new_values = []
        new_probabilities = []

        ### Write your code here ###
        ### as we know, VE works as performing two factor operations: product and marginalizaiton. 
        ### for our multiply method, we have to define the product of two factors
        ### we can do it by multiplying these factors
        ### the crucial thing here is that we have to multiply all factors containing X which is a variable
        ### to be more precise, it exploits the product form of joint distribution

        ### we saw the examples regarding to this, so in this function we have to write the joints as product
        ### and we need to send the variables inside in order

        for i, factor in enumerate(self.new_variables):
            if other in factor[0]:
                new_values.append(i)
                new_probabilities.append(factor)
        if len(new_values)>1:
            for i in reversed(self.values):
                del new_probabilities[i]
        ### Your code ends here ###

        new_factor = Factor(new_variables, new_values, new_probabilities)
        return new_factor

    def sum_out(self, variable):
        '''Sums out 'variable' from the factor.

        Keyword arguments:
        variable -- Name of a variable in the factor.

        Returns:
        self
        '''
        variable_index = self.variables.index(variable)
        new_values = []
        new_probabilities = []

        ### Write your code here ###
        # in this case we have to have for loop for all the factors 
        # this function is used to sum out(basically removing) the variables 
        # as we sum over any variable, we get a function of other variables (we sum over that variable and keep other variables as function)
        # we do it for some iterations, until getting to final sum 

        # to conclude, we have to eliminate the variables in their order, which will derive us the function of them 
        # so we can use this step over and over till last iteration to sum over all variables 

        for i, factor in enumerate(self.variables):
            for j, v in enumerate(factor[0]):
                if v == variable:
                    new_probabilities = factor[0][:j]+factor[0][j+1]
                for entry in factor[1]:
                    entry = list(entry)
                    new_key = tuple(entry[:j]+entry[j+1:])

                    entry[j] = True
                    prob1 = factor[1][tuple(entry)]
                    entry[j]= False
                    prob2 = factor[1][tuple(entry)]
                    prob = prob1+prob2
                    new_values[new_key] = prob
        ### Your code ends here ###

        self.variables.pop(variable_index)
        self.values = new_values
        self.probabilities = new_probabilities
        return self


class ExactInferenceEngine(object):
    def __init__(self, network):
        self.network = network

    def perform_inference(self, query_variable, evidence_variables):
        ''' Calculates the probability distribution 
        P(query_variable|evidence_variables) using variable elimination.
        Assumes that we have only one query variable.

        Keyword arguments:
        query_variable -- The name of the query variable.
        evidence_variables -- A dictionary containing variable names
                              as keys and observed values as values.

        Returns:
        distribution -- A dictionary containing the values of the query
                        variable as keys and the probabilities as values.
        '''

        known_variables = list(evidence_variables.keys())
        known_variables.append(query_variable)
        hidden_variables = []
        for _,var in enumerate(self.network.nodes()):
            if var not in known_variables:
                hidden_variables.append(var)

        variables, dependency_levels = \
        self.__find_dependency_levels(known_variables, hidden_variables)

        #we don't want the query variable to be considered as known
        #in the factor creation process
        known_variables.remove(query_variable)
        factors = self.__create_factors\
        (evidence_variables, variables, dependency_levels)

        current_factor = Factor()
        for i in range(len(hidden_variables)-1, -1, -1):
            ### write your code here ###
            # as multiply and sum_out methods are finished, in this step we have to call those functions
            # so for all the variables, we have to set observed variables to their observed values
            # for each other variables let say Z_i, we have to sum out Z_i
            # we have to multiply the remaining factors 
            # finally we have to normalize by dividing the result factor 
            pass
            ### your code ends here ###

        current_factor = self.__multiply_factors
        (factors, current_factor, dependency_levels, -1)
        alpha = sum(np.array(current_factor.probabilities))

        distribution = dict()
        for i,value in enumerate(current_factor.values):
            distribution[value[0]] = current_factor.probabilities[i] / alpha

        return distribution

    def __find_dependency_levels(self, known_variables, hidden_variables):
        '''Looks for the level at which a term can be
        extracted in front of an inner summation.
        Doing this for each of the variables allows us to
        decompose the summation over the hidden variables appropriately.

        Keyword arguments:
        known_variables -- A list containing evidence variables
                           and the query variable.
        hidden_variables -- A list of hidden variables.

        Returns:
        variables -- A 'numpy.array' of variables in the network.
        dependency_levels -- A 'numpy.array' containing zero-based indices
                             that indicate the level at which we can extract
                             a certain term in front of an inner summation.
                             The indices of this array and the array 'variables'
                             are aligned, such that 'dependency_level[i]'
                             denotes the dependency level of 'variable[i]'.
        '''
        variables = np.array(self.network.nodes())
        dependency_levels = np.zeros(variables.shape, dtype=int)

        #we look for dependencies between the CPTs of the known variables
        #and the hidden variables and assign an appropriate level to them
        for _,variable in enumerate(known_variables):
            variable_index = np.where(variables==variable)[0][0]
            dependency_levels[variable_index] = -1
            for i in range(len(hidden_variables)):
                hidden = hidden_variables[i]
                children = self.network.successors(hidden)
                variable_is_child = len(list(children)) != 0 and variable in children
                if variable_is_child:
                    dependency_levels[variable_index] = i
                    break

        #we look for dependencies between the CPTs of the hidden variables
        #and the other hidden variables and assign an appropriate level to them
        for i,variable in enumerate(hidden_variables):
            variable_index = np.where(variables==variable)[0][0]
            dependency_levels[variable_index] = i
            for j in range(i+1,len(hidden_variables)):
                hidden = hidden_variables[j]
                children = self.network.successors(hidden)
                variable_is_child = len(list(children)) != 0 and variable in children
                if variable_is_child:
                    dependency_levels[variable_index] = j
                    break   

        return variables, dependency_levels

    def __create_factors(self, evidence_variables, variables, dependency_levels):
        '''
        Keyword arguments:
        evidence_variables -- A dictionary containing evidence variables
                              as keys and the observed values as values.
        variables -- A list of variables in the network.
        dependency_levels -- A 'numpy.array' as returned by
                             'self.__find_dependency_levels'.

        Returns:
        factors -- A list of 'Factor' objects ordered in reverse order
                   of the values in 'dependency_levels'.
        '''

        #we are going to create factors from the highest level of dependency
        sorting_indices = np.argsort(dependency_levels)[::-1]
        number_of_variables = len(variables)
        known_variables = list(evidence_variables.keys())
        factors = []
        for i in range(number_of_variables):
            factor_variables = []
            factor_values = []
            factor_probabilities = []
            variable = variables[sorting_indices[i]]

            variable_cardinality = self.network.get_cardinality(variable)
            values = np.linspace(0, variable_cardinality, \
                                 variable_cardinality-1, dtype=int)
            parents = self.network.get_parents(variable)
            if parents == None:
                #the factor has an empty scope if we have
                #an evidence variable that doesn't have any parents
                if variable in known_variables:
                    evidence_value = evidence_variables[variable]
                    value_index = np.where(values == evidence_value)[0]
                    cpt = self.network.get_cpds(variable)
                    factor_values.append(values[value_index])
                    factor_probabilities.append(cpt.get_values()[value_index])
                else:
                    cpt = self.network.get_cpds(variable)
                    factor_variables.append(variable)
                    for i,val in enumerate(values):
                        factor_values.append([val])
                        factor_probabilities.append(cpt.get_values()[i])
            else:
                #if the variable is one of the evidence variables,
                #the factor will only contain the probabilities
                #that correspond to the observed value
                if variable in known_variables:
                    evidence_value = evidence_variables[variable]
                    value_index = np.where(values == evidence_value)[0]
                    cpt = self.network.get_cpds(variable)

                    #we check if any of the variable's parents
                    # is an evidence variable
                    known_parents = []
                    for _,parent in enumerate(parents):
                        if parent in known_variables:
                            known_parents.append(parent)

                    #if none of the parents are known, we just take 
                    #the probabilities that correspond to the variable's 
                    #observed value
                    if len(known_parents) == 0:
                        factor_variables = list(parents)
                        cpt_values = cpt.get_values().flatten()
                        for i in range(np.product(cpt.cardinality)):
                            parent_assignment = list()
                            consistent_assignment = True
                            for var, value in cpt.assignment([i])[0]:
                                if var == variable and value != evidence_value:
                                    consistent_assignment = False
                                    break
                                elif var != variable:
                                    parent_assignment.append(value)

                            if consistent_assignment:
                                factor_values.append(parent_assignment)
                                factor_probabilities.append(cpt_values[i])
                    #if some of the parents are also evidence variables,
                    #we take the probabilities that correspond
                    #to the observed values
                    else:
                        factor_variables = list(set(parents) - set(known_parents))
                        parent_indices = [x for x,val in enumerate(parents)\
                                          if val in known_parents]
                        for i in range(np.product(cpt.cardinality)):
                            parent_assignment = list()
                            consistent_assignment = True
                            for var, value in cpt.assignment([i])[0]:
                                if var == variable and value != evidence_value:
                                    consistent_assignment = False
                                    break
                                elif var != variable:
                                    if value == evidence_variables[var]:
                                        parent_assignment.append(value)
                                    else:
                                        consistent_assignment = False
                                        break

                            if consistent_assignment:
                                factor_values.append(parent_assignment)
                                factor_probabilities.append(cpt_values[i])
                else:
                    cpt = self.network.get_cpds(variable)

                    #we check if any of the variable's parents
                    #is an evidence variable
                    known_parents = []
                    for _,parent in enumerate(parents):
                        if parent in known_variables:
                            known_parents.append(parent)

                    if len(known_parents) == 0:
                        factor_variables = list(parents)
                        factor_variables.insert(0, variable)
                        cpt_values = cpt.get_values().flatten()

                        for i in range(np.product(cpt.cardinality)):
                            value_assignment = list()
                            for var, value in cpt.assignment([i])[0]:
                                value_assignment.append(value)

                            factor_values.append(value_assignment)
                            factor_probabilities.append(cpt_values[i])
                    #if some of the parents are also evidence variables,
                    #we take the probabilities that correspond
                    #to the observed values
                    else:
                        factor_variables = list(set(parents) - set(known_parents))
                        parent_indices = [x for x,val in enumerate(parents) \
                                          if val in known_parents]
                        cpt_values = cpt.get_values().flatten()

                        for i in range(np.product(cpt.cardinality)):
                            value_assignment = list()
                            consistent_assignment = False
                            for var, value in cpt.assignment([i])[0]:
                                if var == variable:
                                    value_assignment.append(value)
                                else:
                                    if value == evidence_variables[var]:
                                        value_assignment.append(value)
                                    else:
                                        consistent_assignment = False
                                        break

                            if consistent_assignment:
                                factor_values.append(value_assignment)
                                factor_probabilities.append(cpt_values[i])
                        factor_variables.insert(0, variable)
            new_factor = Factor(factor_variables, factor_values, factor_probabilities)
            factors.append(new_factor)
        return factors

    def __multiply_factors(self, factors, current_factor, \
                           dependency_levels, current_dependency_level):
        '''Multiplies the factors that are at the
        distribution summation's current level.

        Keyword arguments:
        factors -- A list of factors.
        current_factor -- A factor storing partially computed values;
                          if its list of variables is empty, we are
                          just starting the computations in the network.
        dependency_levels -- A 'numpy.array' as returned by
                             'self.__find_dependency_levels'.
        current_dependency_level -- The distribution summation's current level.

        Returns:
        current_factor -- A factor storing partially computed updated values.
        '''
        number_of_factors_to_multiply = \
        len(np.where(dependency_levels==current_dependency_level)[0])
        if number_of_factors_to_multiply == 1:
            if len(current_factor.probabilities) == 0:
                current_factor = factors[0]
            else:
                current_factor = current_factor.multiply(factors[0])
            factors.pop(0)
        else:
            for i in range(number_of_factors_to_multiply):
                if len(current_factor.probabilities) == 0:
                    current_factor = factors[0]
                else:
                    current_factor = current_factor.multiply(factors[0])
                factors.pop(0)
        return current_factor

### Exercise 2: Markov chain Monte Carlo (MCMC)  sampling [60 points]

Your second task for today is that of implementing Gibbs sampling for performing approximate inference in a Bayesian network.
Gibbs-sampling is the form of MCMC which Prof. Prassler used during the lecture today. You can refer to the example at slide 74.

We are now working with only one class, called *ApproximateInferenceEngine*, which exposes one method, i.e *perform_inference*; compared to the exact inference version, this method takes an additional parameter, namely the number of samples that we want to generate. The class also has two private methods, namely *\_\_generate_gibbs_sample* and *\_\_get_parent_assignment_idx*. *\_\_generate_gibbs_sample* does exactly what the name suggests, i.e. it uses Gibbs sampling for generating a sample from the distribution. *\_\_get_parent_assignment_idx* is an implementation-specific function, on the other hand; we are using *pgmpy* for storing the network and its CPTs and given that *pgmpy* doesn't have a function that returns the conditional probability of a variable for a given assignment to the parent variables, we have implemented this ourselves.

**Your task in this exercise is to implement quite a lot of the functionality of *\_\_generate_gibbs_sample* and the main inference loop in *perform_inference* (which amounts to you calling the appropriate methods with the appropriate parameters)**. Just as above, the code contains quite a lot of comments, so you should in principle be able to figure out what you're supposed to do.

*Hint*: While implementing *\_\_generate_gibbs_sample*, you will have to access the values of the variables and their CPTs, so you should familiarise yourself both with how *pgmpy* stores these and with the meaning of *pgmpy*'s exposed methods. *\_\_get_parent_assignment_idx* does some of the work for you, but you still have to do a few things on your own.

In [3]:
class ApproximateInferenceEngine(object):
    def __init__(self, network):
        self.network = network

    def perform_inference(self, query_variable, evidence_variables, \
                          number_of_samples):
        '''Calculates the probability distribution
        P(query_variable|evidence_variables) using
        Gibbs sampling. Assumes that we have only one query variable.

        Keyword arguments:
        query_variable -- The name of the query variable.
        evidence_variables -- A dictionary containing variable names
                              as keys and observed values as values.
        number_of_samples -- The number of samples that should be used
                             in the sampling process.

        Returns:
        distribution -- A dictionary containing the values of the
                        query variable as keys and the probabilities as values.
        '''
        query_cpt = self.network.get_cpds(query_variable)
        number_of_variable_values = query_cpt.cardinality[0]

        distribution = dict()
        for i in range(number_of_variable_values):
            distribution[i] = 0.

        variable_assignments = dict()

        #we initialise the variables randomly before generating samples
        for _,variable in enumerate(self.network.nodes()):
            if variable in evidence_variables.keys():
                variable_assignments[variable] = evidence_variables[variable]
            else:
                value = np.random.randint(0, network.get_cardinality(variable))
                variable_assignments[variable] = value

        for i in range(number_of_samples):
            ### write your code here ###

            # we need to pick a non-evidence variable X_i randomly 
            # and we need to sample it from P(X_i, x_1, ..., x_i-1, x+1, x_n)
            # then we need to keep all other values 
            # finally get get new sample
            # we have to repeat these steps as much as we want

            pass
            ### your code ends here

        normaliser = 0.
        for key in distribution.keys():
            normaliser = normaliser + distribution[key]

        if normaliser > 1e-10:
            for key in distribution.keys():
                distribution[key] = distribution[key] / normaliser
        return distribution

    def __generate_gibbs_sample(self, variable_assignments, evidence_variables):
        '''Generates a random assignment for the
        non-evidence variables in the network, sampling
        each of them given their Markov blanket.

        Keyword arguments:
        variable_assignments -- A dictionary containing variable names
                                as keys and variable assignments as values.
        evidence_variables -- A dictionary containing variable names
                              as keys and observed values as values.

        Returns:
        variable_assignments -- A dictionary containing variable names
                                as keys and variable assignments as values.
        '''
        variables_to_sample = list(set(self.network.nodes()) - \
                                   set(evidence_variables.keys()))

        # as we have set evidence variables to the observed values 
        # we need to set all other variables to random variables
        for variable in variables_to_sample:
            value_probabilities = dict()
            cpt = self.network.get_cpds(variable).get_values().flatten()
            number_of_values = self.network.get_cardinality(variable)
            for value in range(number_of_values):
                value_probabilities[value] = 1.

            #we calculate the product of the probabilities of the children
            #given their parents if the variable has any children
            if len(self.network.successors(variable)) != 0:
                ### write your code here ###
                # in this case we check if it has any children or not
                # if it has children we multiply the probabiliites of the children 
                # by P(X) = P(X | Parents(X))
                pass
                ### your code ends here ###

            normaliser = 0.

            #we multiply the children probability by the 
            #probability of the current variable given its parents
            #(or by its prior if it has no parents)
            if len(self.network.get_parents(variable)) == 0:
                ### write your code here ###
                # in bayes network, we know that variable is conditionally independent of all others given its Markov blanket
                # so we may write it as P(X_i, MB(X_i))
                # so we need to sample from it
                pass
                ### your code ends here ###
            else:
                ### write your code here ###
                # else returns if it does not have any parents
                # so in this case we will multiply by their prior 
                pass
                ### your code ends here ###

            for _,key in enumerate(value_probabilities.keys()):
                value_probabilities[key] = value_probabilities[key] / normaliser

            # we now generate a value for the variable
            # by sampling from its distribution

            value = -1 
            ### write your code here ###

            # as 

            ### your code ends here ###
            variable_assignments[variable] = value

        return variable_assignments

    def __get_parent_assignment_idx(self, variable, assigned_values):
        '''Returns the assigned values to the parent variables of a given variable.

        Keyword arguments:
        variable -- A string representing a variable in the network.
        assigned_values -- A dictionary containing value assignments to variables.

        Returns:
        parent_value_idx -- An integer representing the CPT value index
                            corresponding to the assigned parent values.
        '''
        if len(self.network.get_parents(variable)) == 0:
            return -1
        else:
            parent_value_idx = -1
            cpt = self.network.get_cpds(variable)
            for i in range(np.product(cpt.cardinality)):
                consistent_assignment = True
                for var, value in cpt.assignment([i])[0]:
                    if value != assigned_values[var]:
                        consistent_assignment = False
                        break
                if consistent_assignment:
                    parent_value_idx = i
                    break
            return parent_value_idx

### Testing your code

We'll test our inference engines using the alarm network from the textbook. The code that creates the network and calls the inference functions for answering two queries - $P(Burglary | JohnCalls = 1, MaryCalls = 1)$ and $P(JohnCalls | MaryCalls = 1)$ - is already implemented in the cell below. What you have to do is thus run this cell after completing the implementation of variable elimination and Gibbs sampling, verifying that the results you obtain are the expected ones:

* $P(Burglary | JohnCalls = 1, MaryCalls = 1) \approx < 0.72, 0.28>$
* $P(JohnCalls | MaryCalls = 1) \approx < 0.82, 0.18>$

*Note*: The sampling code is likely to be a bit slow, so wait for a few seconds after you run the cell.

In [4]:
# creating a test Bayesian network
network = BayesianModel([('Burglary', 'Alarm'), \
                        ('Earthquake', 'Alarm'), \
                        ('Alarm', 'JohnCalls'), \
                        ('Alarm', 'MaryCalls')])

# creating the CPTs of the test Bayesian network
cpd_burglary = TabularCPD(variable='Burglary', variable_card=2, \
                          values=[[0.999], [0.001]])
cpd_earthquake = TabularCPD(variable='Earthquake', variable_card=2, \
                            values=[[0.998], [0.002]])
cpd_alarm = TabularCPD(variable='Alarm', variable_card=2, \
                       values=[[0.999, 0.71, 0.06, 0.05],
                               [0.001, 0.29, 0.94, 0.95]], \
                       evidence=['Burglary', 'Earthquake'],
                       evidence_card=[2, 2])
cpd_john_calls = TabularCPD(variable='JohnCalls', variable_card=2, \
                            values=[[0.95, 0.1], [0.05, 0.9]],
                            evidence=['Alarm'], evidence_card=[2])
cpd_mary_calls = TabularCPD(variable='MaryCalls', variable_card=2, \
                            values=[[0.99, 0.3], [0.01, 0.7]],
                            evidence=['Alarm'], evidence_card=[2])

network.add_cpds(cpd_burglary, cpd_earthquake, cpd_alarm, \
                 cpd_john_calls, cpd_mary_calls)
network.check_model()

exact_inference_engine = ExactInferenceEngine(network)
approximate_inference_engine = ApproximateInferenceEngine(network)

#print(exact_inference_engine)

#############################################
# Test 1: P(Burglary | JohnCalls, MaryCalls)
#############################################
query_variable = 'Burglary'
evidence_variables = {'JohnCalls': 1, 'MaryCalls': 1}

resulting_distribution = exact_inference_engine.perform_inference\
(query_variable, evidence_variables)
print('Exact: P(Burglary | JohnCalls, MaryCalls) =', resulting_distribution)

resulting_distribution = approximate_inference_engine.perform_inference\
(query_variable, evidence_variables, 30000)
print('Approximate: P(Burglary | JohnCalls, MaryCalls) =', resulting_distribution)

###################################
# Test 2: P(JohnCalls | MaryCalls)
###################################
query_variable = 'JohnCalls'
evidence_variables = {'MaryCalls': 1}

resulting_distribution = exact_inference_engine.perform_inference\
(query_variable, evidence_variables)
print('Exact: P(JohnCalls | MaryCalls) =', resulting_distribution)

resulting_distribution = approximate_inference_engine.perform_inference\
(query_variable, evidence_variables, 30000)
print('Approximate: P(JohnCalls | MaryCalls) =', resulting_distribution)

/opt/miniconda3/envs/probabilistic-reasoning/lib/python3.9/site-packages/pgmpy/models/BayesianModel.py:8: FutureWarning: BayesianModel has been renamed to BayesianNetwork. Please use BayesianNewtork class, BayesianModel will be removed in future.
  warnings.warn(


AttributeError: 'function' object has no attribute 'probabilities'